In [2]:
import struct
import datetime as dt
import glob
import os
from collections import namedtuple
import matplotlib.image as mpimg
import cv2

import numpy as np
# import pykitti.utils as utils

In [3]:
# fileName = '/Users/GF/Downloads/data_road_velodyne/testing/velodyne/um_000000.bin'
# with open(fileName, mode='rb') as file: # b is important -> binary
#     fileContent = file.read()
# struct.unpack("iiiii", fileContent[:20])

In [36]:
def get_velo_scans(velo_file):
    """read velodyne [x,y,z,reflectance] scan data from binary files."""
    """parse velodyne binary files into arrays."""
    scan = np.fromfile(velo_file, dtype='float32')
    return scan.reshape((-1, 4))
        
# def velo(data_path):
#         """Generator to read velodyne [x,y,z,reflectance] scan data from binary files."""
#         # Find all the Velodyne files
#         velo_path = os.path.join(
#             data_path, 'velodyne_points', 'data', '*.bin')
#         velo_files = sorted(glob.glob(velo_path))

#         # Subselect the chosen range of frames, if any
# #         if self.frames is not None:
# #             velo_files = [velo_files[i] for i in self.frames]

#         # Return a vgenerator yielding Velodyne scans.
#         # Each scan is a Nx4 array of [x,y,z,reflectance]
#         return get_velo_scans(velo_files)
def sobel(img, thresh_min=0, thresh_max=255): # use the threshold values you found to replace 0 and 255
    #1. convert the image to gray scale
    #2. Gaussian blur the image
    #3. Use Sobel to find derievatives for both X and Y Axis
    #4. Use addweight to combine the results
    #5. Convert each pixel to unint8, then apply threshold to get binary image
    ## TODO
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_blur = cv2.GaussianBlur(img_gray, (5, 5), 0)
    #name your result to be sobelX and sobelY
    sobelX = cv2.Sobel(img_blur,cv2.CV_8U,1,0,3)
    sobelY = cv2.Sobel(img_blur,cv2.CV_8U,0,1,3)
    
    ####
    
    combined = cv2.addWeighted(sobelX, 0.9, sobelY, 0.1, 0)
    #scaled_lap = np.uint8(255*combined/np.max(combined)) #for some reason, scaled_lap was only containing 0s and 2s
    binary_output = np.zeros_like(combined)
    binary_output[(combined >= thresh_min) & (combined <= thresh_max)] = 1
    #for i in range(0,len(scaled_lap)):
    #    for j in range(0,len(scaled_lap[i])):
    #        print(scaled_lap[i][j])

    return binary_output




def findConvCenter(image):
    
    # window settings, can be changed if you want. 
    window_width = 30 
    window_height = rawImg.shape[0] / 10.0 # Break image into 10 vertical layers 
    margin = 80 # How much to shift left or right from last found line segment for searching
    window = np.ones(window_width) # Create our window template that we will use for convolutions
    convThres = 100 # filtered out back ground noise

    #Parameters for finding the start points 
    vertical_ratio = 1.0/2  
    horizontal_ratio = 0.15
    
    
    image = np.uint8(255*image/np.max(image))

    #windowCentroids = [] #Output array that store the (left,right) window centroid positions per level
    leftCentroids = []
    rightCentroids = []

    # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
    # and then np.convolve the vertical image slice with the window template 
    ## TODO
    image_bottom = image[int(image.shape[0]/2):(image.shape[0]-1),:]
    sum_arr = np.sum(image_bottom,axis=0,dtype=np.int32)
    conv_arr = np.convolve(sum_arr,window,'same')
    
    leftCenterX = -1
    rightCenterX = -1
    
    center_coord = image.shape[1]/2
    while(leftCenterX == -1 or rightCenterX == -1):
        max_val = np.argmax(conv_arr)
        if(max_val < center_coord+450 and max_val > center_coord-450):
            if(max_val < center_coord and leftCenterX == -1):
                leftCenterX = max_val
            elif(max_val >= center_coord and rightCenterX == -1):
                rightCenterX = max_val
        conv_arr[max_val] = 0
    #450 pixels to either side of center
    
    leftCenterY = image.shape[0] - (window_height/2)
    rightCenterY = leftCenterY
    ####
    # Add what we found for the first layer
    leftCentroids.append((leftCenterX, leftCenterY))
    rightCentroids.append((rightCenterX, rightCenterY))

    #Count how many empty layers for left and right line
    leftEmptyCount, rightEmptyCount = 0, 0
    
    # Go through each layer looking for max pixel locations
    for level in range(1,(int)(image.shape[0]/window_height)):
        # convolve the window into the vertical slice of the image
        ## TODO
        image_window = image[int(image.shape[0]-(level+1)*window_height):int(image.shape[0]-level*window_height-1),:]
        sum_arr = np.sum(image_window,axis=0,dtype=np.int32)
        conv_arr = np.convolve(sum_arr,window,'same')
        ####
        # Find the best left centroid by using past left center as a reference
        # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
        ## TODO
        left_coord = leftCenterX-margin
        if(left_coord < 0):
            left_coord = 0
        right_coord = leftCenterX+margin
        if(right_coord >= image.shape[1]):
            right_coord = image.shape[1] - 1
        conv_arr_left = conv_arr[left_coord:right_coord]
        newleftCenterX = np.argmax(conv_arr_left)
        if(conv_arr_left[newleftCenterX] < convThres):
            leftCenterX = leftCenterX
        else:
            leftCenterX = newleftCenterX + left_coord
        #print(left_coord,right_coord, leftCenterX, conv_arr_left)
        ####   
        # Find the best right centroid by using past right center as a reference
        #if last layer not found, extend margin and reuse old refrence
        ## TODO
        left_coord = rightCenterX-margin
        if(left_coord < 0):
            left_coord = 0
        right_coord = rightCenterX+margin
        if(right_coord >= image.shape[1]):
            right_coord = image.shape[1] - 1
        conv_arr_right = conv_arr[left_coord:right_coord]
        newrightCenterX = np.argmax(conv_arr_right)
        if(conv_arr_right[newrightCenterX] < convThres):
            rightCenterX = rightCenterX
        else:
            rightCenterX = newrightCenterX + left_coord
        ####
        # Add what we found for that layer
        ## TODO
        #print(leftCenterX, leftCenterY, rightCenterX, rightCenterY)
        leftCenterY = leftCenterY - window_height
        rightCenterY = leftCenterY
        leftCentroids.append((leftCenterX, leftCenterY))
        rightCentroids.append((rightCenterX, rightCenterY))
        ####
    ####
    #return two list of tuples 
    return leftCentroids, rightCentroids

def window_mask(width, height, img_ref, centerX, centerY):
    output = np.zeros_like(img_ref)
    #output[int(img_ref.shape[0]-centerY-height/2):int(img_ref.shape[0]-centerY+height/2),max(0,int(centerX-width/2)):min(int(centerX+width/2),img_ref.shape[1])] = 1
    output[int(centerY-height/2):int(centerY+height/2),max(0,int(centerX-width/2)):min(int(centerX+width/2),img_ref.shape[1])] = 1
    return output


def plotWindows(warped):
    leftCentroids, rightCentroids= findConvCenter(warped)
    
    # Points used to draw all the left and right windows
    r_points = np.zeros_like(warped)
    l_points = np.zeros_like(warped)         
    
    if len(leftCentroids) > 0 and len(rightCentroids) > 0:
        if len(leftCentroids) > 0:
            # Go through each level and draw the windows 
            for level in range(0,len(leftCentroids)):
                # Window_mask is a function to draw window areas
                l_mask = window_mask(window_width,window_height,warped,leftCentroids[level][0],leftCentroids[level][1])
                # Add graphic points from window mask here to total pixels found 
                l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255

        if len(rightCentroids) > 0:
            # Go through each level and draw the windows 
            for level in range(0,len(rightCentroids)):
                # Window_mask is a function to draw window areas
                r_mask = window_mask(window_width,window_height,warped,rightCentroids[level][0],rightCentroids[level][1])
                # Add graphic points from window mask here to total pixels found 
                r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255

        # Draw the results
        template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
        zero_channel = np.zeros_like(template) # create a zero color channel
        template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
        warpage= np.dstack((warped, warped, warped))*255 # making the original road pixels 3 color channels
        output = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results
    #If no window centers found, just display orginal road image
    else:
        output = np.array(cv2.merge((warped,warped,warped)),np.uint8)
    return output


def drawPoly(originImg, warpImg, leftCentroids, rightCentroids, Minv):
    #Generate polynomial functions
    ##TO DO
    h,w = warpImg.shape[:2]
    leftX = np.array([i[0] for i in leftCentroids])
    leftY = np.array([i[1] for i in leftCentroids])
    left_eq = np.polyfit(leftX,leftY,2)
    rightX = np.array([i[0] for i in rightCentroids])
    rightY = np.array([i[1] for i in rightCentroids])
    right_eq = np.polyfit(rightX,rightY,2)
    ####
    
    # Generate y values for plotting
    plotY = np.linspace(0, warpImg.shape[0]-1, warpImg.shape[0])
    # Generate X values for polynomial line 
    ## TO DO
    leftPlotX = ((left_eq[1]*-1) + np.sqrt(np.abs(left_eq[1]**2 - 4*left_eq[0]*(left_eq[2]-plotY)))) / (2 * left_eq[0])
    rightPlotX = ((right_eq[1]*-1) + np.sqrt(np.abs(right_eq[1]**2 - 4*right_eq[0]*(right_eq[2]-plotY)))) / (2 * right_eq[0])
    
    ####
    
    #filter out the out of boundary points
    ##TO DO
    width = warpImg.shape[1]
    leftPlotX[leftPlotX<0] = 0
    leftPlotX[leftPlotX>=width] = width-1
    rightPlotX[rightPlotX<0] = 0
    rightPlotX[rightPlotX>=width] = width-1
    ####
    
    #Generate (X,Y) points for plotting
    leftPts = np.array([np.transpose(np.vstack([leftPlotX, plotY]))])
    rightPts = np.array([np.flipud(np.transpose(np.vstack([rightPlotX, plotY])))])
    points = np.hstack((leftPts, rightPts))
    #plot to warped image
    ##To DO
    #warpedpoints = cv2.warpPerspective(points, Minv, (w,h), flags=cv2.INTER_LINEAR)
    plottedImg = cv2.fillPoly(warpImg,np.int32([points]),255)
    warpedImg = cv2.warpPerspective(plottedImg, Minv, (w,h), flags=cv2.INTER_LINEAR)
    newImg = cv2.addWeighted(originImg, 1, warpedImg, .5, 0)
    ####
    
    #return the new image and the coefficients of two polynomial equations 
    return newImg, leftPlotX, rightPlotX


print ('...')

...


In [64]:
#filename = '/Users/GF/Downloads/data_road_velodyne-2/training/velodyne/um_000003.bin'
#get lidar data from file and put it into a numpy array, also get camera image and nn output
filename = '/Users/Mike/Desktop/data_road_velodyne/training/velodyne/um_000022.bin'
camera_filename = '/Users/Mike/Desktop/data_road_gray/training/image_0/um_000022.png'
nn_filename = '/Users/Mike/OneDrive/School/Illinois/SVN/ece498sm/project/nn_22.png'
cam_img = np.fromfile(camera_filename, dtype='float32') #cv2.imread(camera_filename)
nn_img = np.fromfile(nn_filename, dtype='float32') #cv2.imread(camera_filename)
arr = get_velo_scans(filename)
new_arr = np.full((375,1242,4),(0,0,0,255))
#for each data point, round it to the nearest pixel value and insert it into the img
for point in arr:
    if(point[3] > 0 and point[0] > 0 and point[0] <= 37.4 and abs(point[1]) < 9.95 and point[2] < 0):
        x_coord = int(round(point[1]*20)+200)
        y_coord = int(round(point[0]*10))
        new_arr[374-y_coord][820-x_coord] = (0,int(point[3]*255),0,255)
cv2.imwrite("out22.png", new_arr)
img = np.float32(new_arr)
#filter the image for edge detection
thresh_min1 = 0
thresh_max1 = 255
bw_img = sobel(img, thresh_min1, thresh_max1)
#detect lanes from edge detected image
leftCentroids, rightCentroids = findConvCenter(bw_img)

#cv2.imwrite("out3_bw.png", bw_img)

#warp perspctive points, manually found from picture
x3, y3 = 655, 299
x1, y1 = 657, 243
x4, y4 = 518, 300
x2, y2 = 516, 248

#400,400
#1242,375 .9375

dst = np.float32([(708,260), #109 #279
                  (343,263), #150 101
                  (788,332), #284
                  (165,325)])   
src = np.float32([(x1,y1),(x2,y2),(x3,y3),(x4,y4)])
#create perspective transforms
M = cv2.getPerspectiveTransform(src,dst)
Minv = cv2.getPerspectiveTransform(dst,src)
#warp image
warpedImg = cv2.warpPerspective(img, M, (1242,375), flags=cv2.INTER_LINEAR)
cv2.imwrite("out22_w.png", warpedImg)
#newImg = cv2.addWeighted(cam_img, 1, warpedImg, .5, 0)
#draw detected lane and combine it with camera image or with output of neural network
#imgOut, leftX, rightX = drawPoly(cam_img, img, leftCentroids, rightCentroids, M)
imgOut, leftX, rightX = drawPoly(nn_img, img, leftCentroids, rightCentroids, M)
cv2.imwrite("out22_comb.png", imgOut)
#cv2.imwrite("out3_comb.png", camImg)
        
# scan = np.fromfile(filename, dtype='uint8')
# cv2.imwrite('scan.png', scan[:640000].reshape(64,10000))

True